# Modeling
ML Tasks

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## Input

In [2]:
from sklearn.datasets import load_files

corpus = load_files("../data/")

doc_count = len(corpus.data)
print("Doc count:", doc_count)
assert doc_count is 56, "Wrong number of documents loaded, should be 56 (56 stories)"

Doc count: 56


## Vectorizer

In [3]:
from tokenizer import TextWrangler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

bow_stem = CountVectorizer(strip_accents="ascii", tokenizer=TextWrangler(kind="stem"))
X_bow_stem = bow_stem.fit_transform(corpus.data)

tfidf_stem = TfidfVectorizer(strip_accents="ascii", tokenizer=TextWrangler(kind="stem"))
X_tfidf_stem = tfidf_stem.fit_transform(corpus.data)

[nltk_data] Downloading package punkt to ../nltk/...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to ../nltk/...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to ../nltk/...
[nltk_data]   Package wordnet is already up-to-date!


## Models

In [4]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, NMF

n_topics = 5

lda = LatentDirichletAllocation(n_components=n_topics, 
                                learning_decay=0.5, learning_offset=1.,
                                random_state=23)
lsa = TruncatedSVD(n_components=n_topics, random_state=23)
nmf = NMF(n_components=n_topics, solver="mu", beta_loss="kullback-leibler", alpha=0.1, random_state=23)

In [5]:
lda_params = {"lda__learning_decay": [0.5, 0.7, 0.9],
              "lda__learning_offset": [1., 5., 10.]}

## Pipelines

In [6]:
from sklearn.pipeline import Pipeline

lda_pipe = Pipeline([
    ("bow", bow_stem),
    ("lda", lda)
])

lsa_pipe = Pipeline([
    ("tfidf", tfidf_stem),
    ("lsa", lsa)
])

nmf_pipe = Pipeline([
    ("tfidf", tfidf_stem),
    ("nmf", nmf)
])

## Gridsearch

In [7]:
from sklearn.model_selection import GridSearchCV

lda_model = GridSearchCV(lda_pipe, param_grid=lda_params, cv=5, n_jobs=-1)

## Training

In [8]:
lda_pipe.fit(corpus.data)
nmf_pipe.fit(corpus.data)
lsa_pipe.fit(corpus.data)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...', TruncatedSVD(algorithm='randomized', n_components=5, n_iter=5,
       random_state=23, tol=0.0))])

## Evaluation

In [9]:
print("LDA")
print("Log Likelihood:", lda_pipe.score(corpus.data))

LDA
Log Likelihood: -1281669.2791995243


In [10]:
def show_topics(vectorizer, lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [11]:
topic_keywords = show_topics(vectorizer=bow_stem, lda_model=lda_pipe.named_steps.lda, n_words=15)        

print("LDA")
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

LDA


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,room,hand,com,fac,littl,lady,know,think,hous,way,look,wom,door,mat,tim
Topic 1,know,hand,sir,com,let,room,look,tim,door,say,fac,think,cam,good,littl
Topic 2,com,room,hand,know,fac,tim,day,cas,hous,way,look,lik,think,littl,sir
Topic 3,com,littl,hand,room,look,tim,night,know,good,think,door,hous,ask,op,cas
Topic 4,com,look,know,cas,turn,left,think,littl,young,hand,let,road,day,fac,way


In [12]:
topic_keywords = show_topics(vectorizer=tfidf_stem, lda_model=lsa_pipe.named_steps.lsa, n_words=15)        

print("LSA")
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

LSA


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,com,room,hand,know,look,littl,tim,fac,think,hous,door,let,cas,way,cam
Topic 1,lestrad,colonel,bust,cunningham,mycroft,oldacr,mccarthy,straker,mcfarlane,norwood,alec,cadog,barclay,pap,ross
Topic 2,godfrey,colonel,staunton,emswor,cunningham,barclay,armstrong,straker,hors,old,overton,sir,tregen,ralph,bennet
Topic 3,simon,robert,st,lestrad,godfrey,hors,ferguson,miss,wif,lady,lord,barclay,straker,hosm,sarah
Topic 4,hopkin,lestrad,carey,smi,bust,godfrey,pet,bicyc,stanley,mccarthy,carruth,staunton,hut,oldacr,harpoon


In [13]:
topic_keywords = show_topics(vectorizer=tfidf_stem, lda_model=nmf_pipe.named_steps.nmf, n_words=15)        

print("NMF")
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

NMF


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,wait,way,went,view,tru,NUM,voic,walk,threw,understand,us,window,ask,want,short
Topic 1,triv,revelry,retir,ring,street,venezuel,greek,buzz,return,troph,got,NUM,seen,viol,warm
Topic 2,tap,unpleas,villain,pip,vit,surfac,try,twitch,vary,team,wak,fing,rock,path,perceiv
Topic 3,understood,suggest,tabl,tobacco,tig,undertak,swung,try,sound,urg,tin,story,cheek,trembl,brok
Topic 4,twic,trick,tragedy,waddl,string,saw,ut,suppos,unquest,westminst,pol,turn,instinct,struck,rocky


In [14]:
import pyLDAvis
from pyLDAvis.sklearn import prepare
pyLDAvis.enable_notebook()

prepare(lda_pipe.named_steps.lda, X_bow_stem, bow_stem, mds="tsne")

/home/datadonk23/anaconda3/envs/holmes-topic-models/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2      13.529840 -40.950478       1        1  28.003533
1      87.589325  12.575796       2        1  24.091915
3      85.174118 -40.496067       3        1  22.266089
0      32.787804   8.396887       4        1  18.102037
4      50.990089 -81.165482       5        1   7.536426, topic_info=     Category        Freq      Term       Total  loglift  logprob
term                                                             
6012  Default  162.000000      road  162.000000  30.0000  30.0000
6637  Default  126.000000       son  126.000000  29.0000  29.0000
4051  Default  358.000000      lady  358.000000  28.0000  28.0000
4142  Default  417.000000      left  417.000000  27.0000  27.0000
7536  Default  461.000000      turn  461.000000  26.0000  26.0000
663   Default   39.000000     bicyc   39.000000  25.0000  25.0000
8245  Default  343.000000     young  343.000000  24.0000  24.0000
3259  Default  133.000000       hal  133.000000  23.0000  23.0000
3883  Default  103.000000       jam  103.000000  22.0000  22.0000
2615  Default  157.000000      fath  157.000000  21.0000  21.0000
3129  Default   46.000000      grac   46.000000  20.0000  20.0000
4456  Default   33.000000  mccarthy   33.000000  19.0000  19.0000
6806  Default  271.000000      stat  271.000000  18.0000  18.0000
7413  Default   81.000000     track   81.000000  17.0000  17.0000
2211  Default   40.000000       duk   40.000000  16.0000  16.0000
4261  Default  725.000000      look  725.000000  15.0000  15.0000
1078  Default   29.000000   carruth   29.000000  14.0000  14.0000
3489  Default  113.000000      hors  113.000000  13.0000  13.0000
1087  Default  604.000000       cas  604.000000  12.0000  12.0000
4168  Default  591.000000       let  591.000000  11.0000  11.0000
3452  Default  148.000000      hold  148.000000  10.0000  10.0000
819   Default  116.000000       boy  116.000000   9.0000   9.0000
1398  Default  920.000000       com  920.000000   8.0000   8.0000
4031  Default  782.000000      know  782.000000   7.0000   7.0000
6334  Default  450.000000      shal  450.000000   6.0000   6.0000
5189  Default   90.000000      path   90.000000   5.0000   5.0000
4226  Default  714.000000     littl  714.000000   4.0000   4.0000
6062  Default   40.000000   rucastl   40.000000   3.0000   3.0000
1390  Default  161.000000   colonel  161.000000   2.0000   2.0000
4283  Default  198.000000       lov  198.000000   1.0000   1.0000
...       ...         ...       ...         ...      ...      ...
3259   Topic5   33.842038       hal  133.857889   1.2103  -5.9110
2615   Topic5   34.826066      fath  157.772453   1.0746  -5.8824
3347   Topic5    9.817619       hea   17.278266   2.0202  -7.1486
4070   Topic5    9.782508  landlord   18.263871   1.9611  -7.1521
8245   Topic5   46.397073     young  343.994260   0.5820  -5.5955
5189   Topic5   22.558780      path   90.084975   1.2008  -6.3166
819    Topic5   25.751792       boy  116.388267   1.0770  -6.1842
4142   Topic5   49.149784      left  417.957031   0.4449  -5.5379
7536   Topic5   51.193377      turn  461.249476   0.3871  -5.4971
1398   Topic5   63.634261       com  920.843812  -0.0867  -5.2796
4261   Topic5   57.011233      look  725.264604   0.0421  -5.3895
1087   Topic5   52.542417       cas  604.490262   0.1427  -5.4711
4031   Topic5   55.023444      know  782.236534  -0.0690  -5.4250
6806   Topic5   35.871049      stat  271.509321   0.5614  -5.8528
1786   Topic5   44.704787       day  555.492234   0.0656  -5.6327
7264   Topic5   47.046392     think  657.982012  -0.0526  -5.5816
4168   Topic5   45.301714       let  591.337220   0.0164  -5.6194
6334   Topic5   40.781649      shal  450.571062   0.1831  -5.7245
4226   Topic5   47.037777     littl  714.279222  -0.1349  -5.5818
5425   Topic5   36.772570     point  367.147624   0.2844  -5.8280
7209   Topic5   38.886174       tel  459.456490   0.1160  -5.7721
6

In [15]:
prepare(nmf_pipe.named_steps.nmf, X_tfidf_stem, tfidf_stem, mds="tsne")

/home/datadonk23/anaconda3/envs/holmes-topic-models/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/home/datadonk23/anaconda3/envs/holmes-topic-models/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/home/datadonk23/anaconda3/envs/holmes-topic-models/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/home/datadonk23/anaconda3/envs/holmes-topic-models/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0       80.307037   16.309763       1        1  53.959129
3      108.704819  -70.961517       2        1  12.307049
1       -5.732002  -23.836254       3        1  11.971417
2       -6.824123 -115.339539       4        1  11.817915
4       78.230980 -157.529938       5        1   9.944491, topic_info=     Category      Freq        Term     Total  loglift  logprob
term                                                           
7545  Default  1.000000        twic  1.000000  30.0000  30.0000
7626  Default  1.000000  understood  1.000000  29.0000  29.0000
7018  Default  1.000000     suggest  1.000000  28.0000  28.0000
7490  Default  1.000000        triv  1.000000  27.0000  27.0000
5362  Default  1.000000         pip  1.000000  26.0000  26.0000
7897  Default  1.000000     villain  1.000000  25.0000  25.0000
7723  Default  1.000000     unpleas  1.000000  24.0000  24.0000
7177  Default  1.000000         tap  1.000000  23.0000  23.0000
7516  Default  2.000000         try  2.000000  22.0000  22.0000
7915  Default  1.000000         vit  1.000000  21.0000  21.0000
7476  Default  1.000000       trick  1.000000  20.0000  20.0000
6918  Default  1.000000      street  1.000000  19.0000  19.0000
5957  Default  1.000000     revelry  1.000000  18.0000  18.0000
6000  Default  1.000000        ring  1.000000  17.0000  17.0000
5946  Default  1.000000       retir  1.000000  16.0000  16.0000
7067  Default  1.000000      surfac  1.000000  15.0000  15.0000
7363  Default  1.000000     tobacco  1.000000  14.0000  14.0000
7152  Default  1.000000        tabl  1.000000  13.0000  13.0000
7422  Default  1.000000     tragedy  1.000000  12.0000  12.0000
7955  Default  0.000000         wak  0.000000  11.0000  11.0000
6157  Default  1.000000         saw  1.000000  10.0000  10.0000
7552  Default  1.000000      twitch  1.000000   9.0000   9.0000
7849  Default  0.000000    venezuel  0.000000   8.0000   8.0000
2697  Default  1.000000        fing  1.000000   7.0000   7.0000
7060  Default  1.000000      suppos  1.000000   6.0000   6.0000
7202  Default  0.000000        team  0.000000   5.0000   5.0000
7835  Default  1.000000        vary  1.000000   4.0000   4.0000
7133  Default  0.000000       swung  0.000000   3.0000   3.0000
5430  Default  0.000000         pol  0.000000   2.0000   2.0000
7536  Default  1.000000        turn  1.000000   1.0000   1.0000
...       ...       ...         ...       ...      ...      ...
7555   Topic5  0.209480       tying  0.225542   2.2343  -6.3257
6944   Topic5  0.674130      string  0.841559   2.0863  -5.1569
6606   Topic5  0.214903        soak  0.234905   2.2192  -6.3001
7545   Topic5  1.056382        twic  1.538478   1.9322  -4.7077
7476   Topic5  0.826547       trick  1.160359   1.9689  -4.9530
3897   Topic5  0.224899    jealousy  0.251130   2.1978  -6.2546
3780   Topic5  0.515722    instinct  0.671548   2.0441  -5.4247
7440   Topic5  0.182345      transp  0.196362   2.2341  -6.4644
691    Topic5  0.160971    blackhea  0.170070   2.2532  -6.5891
7422   Topic5  0.764710     tragedy  1.257867   1.8105  -5.0308
2833   Topic5  0.409240        forg  0.608730   1.9111  -5.6560
1310   Topic5  0.321207      clench  0.446683   1.9784  -5.8982
6157   Topic5  0.668001         saw  1.325388   1.6230  -5.1660
8140   Topic5  0.456510         wis  0.807664   1.7376  -5.5467
5430   Topic5  0.524199         pol  0.996382   1.6659  -5.4084
7810   Topic5  0.580005          ut  1.200445   1.5807  -5.3073
6962   Topic5  0.513422      struck  1.012696   1.6289  -5.4292
7060   Topic5  0.553723      suppos  1.231362   1.5089  -5.3536
7670   Topic5  0.474147    unfortun  1.068491   1.4957  -5.5088
3459   Topic5  0.415423         hom  0.899370   1.5358  -5.6410
4136   Topic5  0.443932         led  1.038162   1.4586  -5.5746
7014   Topic5  0.406652        suff  0.877637   1.5389  -5.6623
6676   Topic5  0.354953        